# About this script
This script is used for testing how to improve speed of updating files with multiprocess.
The test shows great improvement
And at last the results are identical between traditional for loop and multiprocess.


In [1]:
import os
import Paths
import datetime
import pandas as pd

In [2]:
# list all sub folders in the given folder
def list_subfolders(folder):
	return [f.path for f in os.scandir(folder) if f.is_dir()]

update_folders = list_subfolders(Paths.Fundamental_Data_Update_Folder)
update_folders.sort()
update_folders

['./Data/Fundamental/Updates/2021-11-23',
 './Data/Fundamental/Updates/2021-11-25',
 './Data/Fundamental/Updates/2021-11-26']

In [3]:
def update_csv(csv_file):
	if csv_file in production_csv_files:
				#print(csv_file + " already in production folder, updating.")
			# append the update csv file to the production csv file
			df_update = pd.read_csv(os.path.join(folder, csv_file))
			# Add new column to the dataframe with value, this is the update date stamp
			df_update['Update_Date'] =update_time_stamp 

			#For the production files:
			df_production = pd.read_csv(os.path.join(Paths.Fundamental_Data_Production_Folder, csv_file))
			df_production = df_production.append(df_update)
			df_production.to_csv(os.path.join(Paths.Fundamental_Data_Production_Folder, csv_file), index=False)
			#	print(csv_file + " updated.")
	else:
		# create new csv file in the production folder with the update date stamp
			#print(csv_file + " not in production folder, creating new file.")
		df_update = pd.read_csv(os.path.join(folder, csv_file))
		# Add new column to the dataframe with value, this is the update date stamp
		df_update['Update_Date'] =update_time_stamp
		df_update.to_csv(os.path.join(Paths.Fundamental_Data_Production_Folder, csv_file), index=False)
			#print(csv_file + " created.")

In [4]:
os.cpu_count()

12

In [5]:
def multi_update():
	#Using multiprocessing to speed up the process
	pool = Pool(os.cpu_count())
	pool.map(update_csv, update_csv_files)
	pool.close()
	pool.join()


In [6]:
from multiprocessing import Pool
import time
start_time = time.time()

for folder in update_folders:
	# Get update folder names from path, it will be a update date stamp.
	update_time_stamp= folder.split('/')[-1]
	print("Checking update folder: " + update_time_stamp)
	# Get files from folder which ends with csv and store them into a list
	update_csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]
	production_csv_files = [f for f in os.listdir(Paths.Fundamental_Data_Production_Folder) if f.endswith('.csv')]

	print("Start Multiprocessing for folder: " + update_time_stamp)

	multi_update()
	
	print("Finished Multiprocessing for folder: " + update_time_stamp)
	#Move the update folder and all its files to the archive folder
	os.rename(folder, os.path.join(Paths.Fundamental_Data_Archived_Folder, update_time_stamp))
	print("Update folder moved to archive folder.")

print("--- %s seconds ---" % (time.time() - start_time))

Checking update folder: 2021-11-23
Start Multiprocessing for folder: 2021-11-23
Finished Multiprocessing for folder: 2021-11-23
Update folder moved to archive folder.
Checking update folder: 2021-11-25
Start Multiprocessing for folder: 2021-11-25
Finished Multiprocessing for folder: 2021-11-25
Update folder moved to archive folder.
Checking update folder: 2021-11-26
Start Multiprocessing for folder: 2021-11-26
Finished Multiprocessing for folder: 2021-11-26
Update folder moved to archive folder.
--- 59.00800704956055 seconds ---


In [14]:
# Compare if two directories contians the same files
def compare_dir(dir1, dir2):
	dir1_files = [f for f in os.listdir(dir1) if f.endswith('.csv')]
	dir2_files = [f for f in os.listdir(dir2) if f.endswith('.csv')]
	return set(dir1_files) == set(dir2_files)

dir2 = Paths.Fundamental_Data_Production_Folder + "-For-Loop"
difference = compare_dir(Paths.Fundamental_Data_Production_Folder, dir2)
difference

True

In [15]:
# Check if two csv files are identical
def compare_csv(file1, file2):
	df1 = pd.read_csv(os.path.join(Paths.Fundamental_Data_Production_Folder, file1))
	df2 = pd.read_csv(os.path.join(Paths.Fundamental_Data_Production_Folder, file2))
	return df1.equals(df2)

loop_files = [f for f in os.listdir(dir2) if f.endswith('.csv')]
mp_files = [f for f in os.listdir(Paths.Fundamental_Data_Production_Folder) if f.endswith('.csv')]

diff_list = []
for lp_file in loop_files:
	if lp_file in mp_files:
		lp_df = pd.read_csv(os.path.join(dir2, lp_file))
		mp_df = pd.read_csv(os.path.join(Paths.Fundamental_Data_Production_Folder, lp_file))
		if lp_df.equals(mp_df):
			# Do nothing
			pass
		else:
			print(lp_file + " is not in the multiprocessing folder.")
			diff_list.append(lp_file)

In [16]:
diff_list

[]